# Análise dos logs de execução do exportdoi

A geração dos arquivos log é realizada à partir da execução dos comandos:

**DOIManager-BRA/exportdoi:**
```bash
    $ rancher --env SCIELO-PROCESSING logs --follow --tail 0 $(rancher --env SCIELO-PROCESSING ps | grep 'DOIManager-BRA/exportdoi ' | cut -d ' ' -f1) 2> `date +%Y%m%d`-exportdoi.log
```

**DOIManager-BRA/dashboard:**
Dependendo de como a ordem de depósito for executada, os eventos tipo ``enqueued deposit for "..."`` serão emitidos pelo processo do *dashboard* ou do *exportdoi*. Para fins da nossa análise as mensagens de ambos serão direcionadas para o arquivo `` `date +%Y%m%d`-exportdoi.log ``.
```bash
    $ rancher --env SCIELO-PROCESSING logs --follow --tail 0 $(rancher --env SCIELO-PROCESSING ps | grep 'DOIManager-BRA/dashboard ' | cut -d ' ' -f1) > `date +%Y%m%d`-exportdoi.log
```

**DOIManager-BRA/celeryworker:**
```bash
    $ rancher --env SCIELO-PROCESSING logs --follow --tail 0 $(rancher --env SCIELO-PROCESSING ps | grep 'DOIManager-BRA/celeryworker ' | cut -d ' ' -f1) 2> `date +%Y%m%d`-celeryworker.log
```

## Análise do dia 2018-07-25

In [1]:
import re

In [2]:
with open('20180725-exportdoi.log') as exportdoi:
    pids = []
    for line in exportdoi:
        match = re.search(r'scl_S[0-9]{4}-[0-9Xx-]{17}', line)
        if match:
            pids.append(match.group()[4:])

In [3]:
with open('20180725-pids.txt', 'w') as pidsfile:
    for pid in pids: 
        pidsfile.write(pid + '\n')
pids

['S0001-37652018000301175',
 'S0034-76122018000200321',
 'S1516-44462018000200192',
 'S0001-37652018000301035',
 'S0034-76122018000200221',
 'S1516-44462018000200227',
 'S0001-37652018000300993',
 'S0034-76122018000200264',
 'S1516-44462018000200138',
 'S0001-37652018000301215',
 'S0102-33062018005006102',
 'S1516-44462018000200220',
 'S0001-37652018000301043',
 'S0034-76122018000200244',
 'S1516-44462018000200229',
 'S0001-37652018000301073',
 'S0102-33062018005006101',
 'S1516-44462018000200212',
 'S0001-37652018000300991',
 'S0102-33062018005006103',
 'S1516-44462018000200115',
 'S0001-37652018000301187',
 'S0034-76122018000200285',
 'S1516-44462018000200226',
 'S0001-37652018000301131',
 'S0101-28002018005013104',
 'S1516-44462018000200154',
 'S0001-37652018000301251',
 'S0102-33062018005006104',
 'S1516-44462018000200145',
 'S0001-37652018000301101',
 'S0034-76122018000200199',
 'S1516-44462018000200174',
 'S0001-37652018000301233',
 'S0034-76122018000200303',
 'S1516-444620180002

In [4]:
print('Total de PIDs elencados para depósito em 2018-07-25:', len(pids))
assert len(pids) == 5319  # 5319 é o total exibido no log do exportdoi

Total de PIDs elencados para depósito em 2018-07-25: 5319


### Análise dos logs de execução do celeryworker

In [5]:
with open('20180725-celeryworker.log') as celeryworker:
    queued_pids = []
    for line in celeryworker:
        match = re.search(r'enqueued deposit for "scl_S[0-9]{4}-[0-9Xx-]{17}"', line)
        if match:
            queued_pids.append(match.group())

In [6]:
queued_pids

['enqueued deposit for "scl_S1516-44462018000200227"',
 'enqueued deposit for "scl_S0001-37652018000301035"',
 'enqueued deposit for "scl_S0034-76122018000200321"',
 'enqueued deposit for "scl_S1516-44462018000200138"',
 'enqueued deposit for "scl_S0102-33062018005006102"',
 'enqueued deposit for "scl_S0001-37652018000301175"',
 'enqueued deposit for "scl_S0034-76122018000200264"',
 'enqueued deposit for "scl_S0001-37652018000301215"',
 'enqueued deposit for "scl_S1516-44462018000200220"',
 'enqueued deposit for "scl_S0034-76122018000200244"',
 'enqueued deposit for "scl_S0001-37652018000301043"',
 'enqueued deposit for "scl_S0001-37652018000300993"',
 'enqueued deposit for "scl_S0001-37652018000300991"',
 'enqueued deposit for "scl_S1516-44462018000200212"',
 'enqueued deposit for "scl_S1516-44462018000200192"',
 'enqueued deposit for "scl_S0102-33062018005006101"',
 'enqueued deposit for "scl_S1516-44462018000200229"',
 'enqueued deposit for "scl_S0001-37652018000301073"',
 'enqueued

In [7]:
print('Total de PIDs enfileirados para depósito em 2018-07-25:', len(queued_pids))
assert len(queued_pids) == 5319  # 5319 é o total exibido no log do exportdoi

Total de PIDs enfileirados para depósito em 2018-07-25: 5319


In [8]:
with open('20180725-celeryworker.log') as celeryworker:
    registry_dispatcher_document_started_pids = []
    registry_dispatcher_document_finished_pids = []
    for line in celeryworker:
        match_start = re.search(r'''started the `registry_dispatcher_document` stage for "\('S[0-9]{4}-[0-9Xx-]{17}', 'scl'\)"''', line)
        if match_start:
            registry_dispatcher_document_started_pids.append(match_start.group())
        match_finish = re.search(r'''finished the `registry_dispatcher_document` stage for "\('S[0-9]{4}-[0-9Xx-]{17}', 'scl'\)"''', line)
        if match_finish:
            registry_dispatcher_document_finished_pids.append(match_finish.group())

In [9]:
registry_dispatcher_document_started_pids

['started the `registry_dispatcher_document` stage for "(\'S0034-76122018000200264\', \'scl\')"',
 'started the `registry_dispatcher_document` stage for "(\'S1516-44462018000200227\', \'scl\')"',
 'started the `registry_dispatcher_document` stage for "(\'S0034-76122018000200321\', \'scl\')"',
 'started the `registry_dispatcher_document` stage for "(\'S0001-37652018000301035\', \'scl\')"',
 'started the `registry_dispatcher_document` stage for "(\'S0001-37652018000301175\', \'scl\')"',
 'started the `registry_dispatcher_document` stage for "(\'S1516-44462018000200138\', \'scl\')"',
 'started the `registry_dispatcher_document` stage for "(\'S0102-33062018005006102\', \'scl\')"',
 'started the `registry_dispatcher_document` stage for "(\'S0001-37652018000301215\', \'scl\')"',
 'started the `registry_dispatcher_document` stage for "(\'S1516-44462018000200220\', \'scl\')"',
 'started the `registry_dispatcher_document` stage for "(\'S0001-37652018000301043\', \'scl\')"',
 'started the `regis

In [10]:
print('Total de PIDs que passaram pelo estágio "registry_dispatcher_document":', len(registry_dispatcher_document_started_pids))
assert len(registry_dispatcher_document_started_pids) == len(registry_dispatcher_document_finished_pids)

Total de PIDs que passaram pelo estágio "registry_dispatcher_document": 464


A task *registry_dispatcher_document* é responsável por inserir o artigo no pipeline de depósito. Dos 5319, apenas 464 artigos foram processados pela task, o que já nos mostra indícios de problemas. A fim de nos ajudar na visualização das mensagens de log, vamos agrupar as mensagens por PID:

In [11]:
with open('20180725-celeryworker.log') as celeryworker:
    messages_by_pid = {}
    for line in celeryworker:
        match = re.search(r'S[0-9]{4}-[0-9Xx-]{17}', line)
        if match:
            _pid = messages_by_pid.setdefault(match.group(), [])
            _pid.append(line)

In [12]:
len(messages_by_pid)

5319

Vamos listar a frequência das mensagens por PID:

In [13]:
_freq = {}
for _pid, _msgs in messages_by_pid.items():
    print(_pid, len(_msgs))
    _occ = _freq.setdefault(len(_msgs), [])
    _occ.append(1)
    
_freq = {count: len(freq) for count, freq in _freq.items()}

S1516-44462018000200227 7
S0001-37652018000301035 7
S0034-76122018000200321 7
S1516-44462018000200138 7
S0102-33062018005006102 7
S0001-37652018000301175 7
S0034-76122018000200264 7
S0001-37652018000301215 7
S1516-44462018000200220 7
S0034-76122018000200244 7
S0001-37652018000301043 7
S0001-37652018000300993 7
S0001-37652018000300991 7
S1516-44462018000200212 7
S1516-44462018000200192 7
S0102-33062018005006101 7
S1516-44462018000200229 7
S0001-37652018000301073 7
S0034-76122018000200221 7
S0102-33062018005006103 7
S0001-37652018000301187 7
S1516-44462018000200115 7
S1516-44462018000200154 7
S0001-37652018000301251 7
S1516-44462018000200226 7
S0034-76122018000200285 7
S1516-44462018000200145 7
S0101-28002018005013104 7
S0001-37652018000301131 7
S0102-33062018005006104 7
S0001-37652018000301233 7
S0034-76122018000200199 7
S1516-44462018000200174 7
S0001-37652018000301059 7
S0001-37652018000301101 7
S0034-76122018000200303 7
S1516-44462018000200181 7
S0001-37652018000301273 7
S1516-444620

S0104-59702018000200617 1
S1414-462X2018000100076 1
S0104-59702018000200515 1
S1414-462X2018000100063 1
S0101-47142018000100303 1
S1981-67232018000100444 1
S0102-71822018000100213 1
S0104-59702018000200615 1
S1415-43662018000500326 1
S0102-71822018000100207 1
S0104-59702018000200311 1
S0104-59702018000200469 1
S0102-71822018000100200 1
S1983-14472017000400418 1
S0104-59702018000200595 1
S0037-86822018005001101 1
S1415-43662018000500349 1
S0104-59702018000200353 1
S1415-43662018000500320 1
S0102-71822018000100100 1
S0104-59702018000200429 1
S0102-69092018000200704 1
S0074-02762018000800402 1
S1415-43662018000500301 1
S0102-71822018000100212 1
S1415-43662018000500360 1
S0102-71822018000100800 1
S0104-59702018000200613 1
S1981-67232018000100446 1
S0104-59702018000200449 1
S1414-462X2018000100084 1
S0102-71822018000100211 1
S1414-462X2018000100092 1
S0104-59702018000200591 1
S0104-59702018000200601 1
S0102-71822018000100209 1
S0303-76572018000100203 1
S0102-71822018000100202 1
S1516-143920

S0034-71672018000300992 1
S0034-71672018000300967 1
S1517-106X2018000100075 1
S0034-71672018000301194 1
S1806-66902018000300381 1
S0034-71672018000301013 1
S0034-71672018000301055 1
S1806-66902018000300437 1
S1517-106X2018000100191 1
S2175-62362018000300777 1
S1806-48922013000100091 1
S0034-71672018000301092 1
S0034-71672018000301099 1
S1676-24442018000200109 1
S1413-78522018000100048 1
S0034-71672018000301038 1
S0103-51502018000100220 1
S1413-78522018000100036 1
S0103-51502018000100301 1
S0103-51502018000100302 1
S1413-78522018000100019 1
S1676-24442018000200083 1
S1676-24442018000200068 1
S1676-24442018000200105 1
S1413-78522018000100030 1
S0103-51502018000100210 1
S1676-24442018000200092 1
S0103-51502018000100214 1
S1413-78522018000100027 1
S0103-51502018000100218 1
S1676-24442018000200116 1
S1413-78522018000100072 1
S1676-24442018000200111 1
S1676-24442018000200099 1
S1413-78522018000100016 1
S0103-51502018000100215 1
S1676-24442018000200095 1
S1413-78522018000100022 1
S0103-515020

S0101-31572018000200377 1
S2176-45732018000200002 1
S2176-45732018000200139 1
S1414-98932018000200301 1
S0101-31572018000200324 1
S1676-06032018000300501 1
S0101-31572018000200304 1
S0100-29452018000203002 1
S2176-45732018000200006 1
S0100-29452018000200905 1
S1414-98932018000200191 1
S0101-31572018000200280 1
S0100-29452018000200403 1
S1413-78522018000300187 1
S1413-78522018000300191 1
S1413-65382018000200173 1
S1413-65382018000200161 1
S1984-22012018000100212 1
S1414-98932018000200378 1
S1413-65382018000200261 1
S1413-78522018000300198 1
S1413-78522018000300175 1
S1984-22012018000100211 1
S1413-65382018000200247 1
S1414-98932018000200332 1
S0101-31572018000200358 1
S1414-98932018000200249 1
S1413-65382018000200199 1
S2176-45732018000200155 1
S1413-78522018000300206 1
S1413-78522018000300170 1
S1413-78522018000300154 1
S0101-31572018000200237 1
S1414-98932018000200363 1
S1413-65382018000200277 1
S1414-98932018000200262 1
S1414-98932018000200347 1
S1413-78522018000300166 1
S1413-785220

S1516-44462018005005101 1
S1516-44462018005005102 1
S1516-44462018005005108 1
S1516-44462018005005104 1
S1414-32832018005011102 1
S1676-06032018000300304 1
S2236-89062018000200225 1
S2236-89062018000200314 1
S2236-89062018000200323 1
S2236-89062018000200184 1
S1414-32832018005011101 1
S2446-47402018000200102 1
S2446-47402018000200138 1
S2446-47402018000200157 1
S1516-14392018000500203 1
S2236-89062018000200352 1
S1414-81452018000200213 1
S2236-89062018000200238 1
S2446-47402018000200166 1
S2446-47402018000200093 1
S1516-14392018000800214 1
S2446-47402018000200147 1
S2236-89062018000200307 7
S2446-47402018000200176 1
S2236-89062018000200173 1
S2236-89062018000200192 1
S2446-47402018000200127 1
S1676-06032018000300206 1
S1676-06032018000300303 1
S2236-89062018000200202 1
S2446-47402018000200110 1
S2236-89062018000200348 1
S2236-89062018000200159 1
S2446-47402018000200115 1
S1984-77262017000400437 1
S0034-75902018000300267 1
S2448-167X2018000300457 1
S0034-75902018000300244 1
S0102-311X20

S0085-56262018000200125 1
S0085-56262018000200135 1
S1677-55382018000300585 1
S1677-55382018000300629 1
S0085-56262018000200090 1
S0085-56262018000200112 1
S1677-55382018000300524 1
S0085-56262018000200097 1
S1677-55382018000300647 1
S1677-55382018000300591 1
S0085-56262018000200119 1
S1677-55382018000300555 1
S1677-55382018000300608 1
S0085-56262018000200148 1
S1677-55382018000300653 1
S1677-55382018000300649 1
S1677-55382018000300429 1
S1677-55382018000300422 1
S1677-55382018000300651 1
S1677-55382018000300623 1
S1677-55382018000300433 1
S1677-55382018000300435 1
S1677-55382018000300550 1
S1677-55382018000300475 1
S1677-55382018000300634 1
S1677-55382018000300536 1
S2237-26602018000300522 1
S1677-55382018000300467 1
S1677-55382018000300655 1
S2237-26602018000300441 1
S2237-26602018000300421 1
S2237-26602018000300592 1
S2237-26602018000300563 1
S2237-26602018000300555 1
S2237-26602018000300615 1
S2237-26602018000300539 1
S2237-26602018000300469 1
S2237-26602018000300393 1
S0100-204X20

S0101-20612018000200318 1
S0101-20612018000200263 1
S0100-736X2018000400773 1
S0101-20612018000200293 1
S1679-62252018000200209 1
S0103-64402018000300301 1
S0101-20612018000200335 1
S1679-62252018000200206 1
S1679-62252018000200207 1
S0103-64402018000300309 1
S0100-67622018000200206 1
S0100-879X2018000900602 1
S1679-62252018000200208 1
S1984-77262018000100059 1
S0101-32892018000200170 1
S0101-32892018000200156 1
S1984-77262018000100080 1
S0101-32892018000200197 1
S0100-06832018000100524 1
S1808-18512018000200158 1
S0101-32892018000200131 1
S1982-21702018000200171 1
S0101-32892018000200205 1
S0100-06832018000100522 1
S1516-14392018000800215 1
S1984-77262018000100005 1
S1982-21702018000200186 1
S0100-06832018000100307 1
S1808-18512018000200124 1
S1984-77262018000100149 1
S0100-67622018000200203 1
S0101-32892018000200177 1
S1808-18512018000200110 1
S1414-81452018000400201 1
S0101-32892018000200123 1
S0101-32892018000200146 1
S0101-32892018000200109 1
S0101-32892018000200117 1
S0100-068320

S0102-311X2014001202607 1
S0102-311X2014001202643 1
S0103-507X2018000200233 1
S0103-507X2018000200153 1
S1679-78252018000500510 1
S1415-98482017000100208 1
S0102-311X2014001202491 1
S1806-83242018000101002 1
S0102-311X2014001202669 1
S0103-507X2018000200195 1
S1415-98482017000100207 1
S0103-507X2018000200226 1
S0102-311X2014001202487 1
S0102-311X2014001200001 1
S1806-83242018000100249 1
S0101-41612018000200229 1
S0102-311X2014001202691 1
S1679-78252018000700500 1
S0031-10492018000100226 1
S1519-69842018000400636 1
S1519-69842018000400661 1
S0102-311X2014001202697 1
S0103-507X2018000200201 1
S0101-41612018000200339 1
S0031-10492018000100223 1
S1519-69842018000400750 1
S0102-311X2014001202679 1
S0102-311X2014001202544 1
S0102-311X2014001202545 1
S1980-65742018000200311 1
S0101-41612018000200251 1
S1519-69842018000400619 1
S0102-311X2014001202547 1
S0031-10492018000100225 1
S1414-753X2018000100309 1
S0101-41612018000200215 1
S0102-311X2014001202549 1
S1519-69842018000400644 1
S0103-507X20

S2595-31922018000200134 1
S2595-31922018000200103 1
S2595-31922018000200116 1
S1413-81232018000702133 1
S1983-21172018000100218 1
S2595-31922018000200163 1
S1413-81232018000702443 1
S2359-56472018005006104 1
S1413-81232018000702078 1
S2359-56472018005006103 1
S2359-56472018005006101 1
S2359-56472018005006102 1
S0104-026X2018000200219 1
S0104-026X2018000200603 1
S2317-15372018000200164 1
S0073-47212018000100228 1
S0001-37652018005012105 1
S2317-15372018000200146 1
S0001-37652018005012106 1
S0104-026X2018000200602 1
S2317-15372018000200173 1
S0104-026X2018000200221 1
S0001-37652018005012103 1
S2317-15372018000200135 1
S2317-15372018000200118 1
S0104-026X2018000200220 1
S0034-737X2018000300286 1
S0034-737X2018000300261 1
S2317-15372018000200193 1
S0103-84782018000600352 1
S0100-29452018000400301 1
S2317-15372018000200156 1
S2317-15372018000200185 1
S0034-737X2018000300234 1
S2317-15372018000200179 1
S0001-37652018005012107 1
S0001-37652018005012109 1
S2317-15372018000200127 1
S1517-707620

S1517-70762018000200401 1
S0102-77862018000100131 1
S0102-77862018000100083 1
S0102-09352018000300850 1
S1517-70762018000200417 1
S0102-76382018000300211 1
S0104-71831997000600135 1
S0103-84782018000800251 1
S1517-70762018000200508 1
S0103-636X2018000200696 1
S0103-84782018000700501 1
S0103-84782018000700351 1
S1517-70762018000200499 1
S0102-77862018000200353 1
S1517-70762018000200201 1
S0104-71831997000600111 1
S1517-70762018000200520 1
S0102-09352018000300946 1
S0103-84782018000700652 1
S0366-69132018000300431 1
S0103-84782018000700930 1
S1517-70762018000200510 1
S0103-636X2018000200480 1
S1517-70762018000200408 1
S0104-71831997000600294 1
S0102-09352018000300661 1
S1517-70762018000200428 1
S0102-76382018000300224 1
S0102-09352018000300993 1
S1517-70762018000200506 1
S0103-84782018000700452 1
S0103-636X2018000200375 1
S0102-76382018000300309 1
S1517-70762018000200419 1
S1517-70762018000200421 1
S0103-636X2018000200653 1
S1517-70762018000200422 1
S1517-70762018000200426 1
S0102-778620

In [14]:
for _msg_no, _occ in _freq.items():
    print('ocorrências com', _msg_no, 'mensagens:\t', _occ)

ocorrências com 7 mensagens:	 452
ocorrências com 1 mensagens:	 4855
ocorrências com 6 mensagens:	 12


## Análise do dia 2018-07-27

In [15]:
with open('20180727-exportdoi.log') as exportdoi:
    pids = []
    for line in exportdoi:
        match = re.search(r'scl_S[0-9]{4}-[0-9Xx-]{17}', line)
        if match:
            pids.append(match.group()[4:])

In [16]:
with open('20180727-pids.txt', 'w') as pidsfile:
    for pid in pids: 
        pidsfile.write(pid + '\n')
pids

['S0001-37652018000301175',
 'S0034-76122018000200321',
 'S1516-44462018000200192',
 'S0001-37652018000301035',
 'S0034-76122018000200221',
 'S1516-44462018000200227',
 'S0001-37652018000300993',
 'S0034-76122018000200264',
 'S1516-44462018000200138',
 'S0001-37652018000301215',
 'S0102-33062018005006102',
 'S1516-44462018000200220',
 'S0001-37652018000301043',
 'S0034-76122018000200244',
 'S1516-44462018000200229',
 'S0001-37652018000301073',
 'S0102-33062018005006101',
 'S1516-44462018000200212',
 'S0001-37652018000300991',
 'S0102-33062018005006103',
 'S1516-44462018000200115',
 'S0001-37652018000301187',
 'S0034-76122018000200285',
 'S1516-44462018000200226',
 'S0001-37652018000301131',
 'S0101-28002018005013104',
 'S1516-44462018000200154',
 'S0001-37652018000301251',
 'S0102-33062018005006104',
 'S1516-44462018000200145',
 'S0001-37652018000301101',
 'S0034-76122018000200199',
 'S1516-44462018000200174',
 'S0001-37652018000301233',
 'S0034-76122018000200303',
 'S1516-444620180002

In [17]:
print('Total de PIDs elencados para depósito em 2018-07-27:', len(pids))
assert len(pids) == 5319  # 5319 é o total exibido no log do exportdoi do dia 2018-07-25

Total de PIDs elencados para depósito em 2018-07-27: 5319


### Análise dos logs de execução do exportdoi/dashboard
Na análise anterior esses eventos estavam sendo emitidos pelo *celeryworker*. Esta mudança é resultado do commit [https://github.com/scieloorg/doi_request/commit/b6d3310e84ac0fcc5336a871afbcc9fe27f29ef8] que move a funcionalidade da task para uma função em outro módulo.

In [18]:
with open('20180727-exportdoi.log') as celeryworker:
    queued_pids = []
    for line in celeryworker:
        match = re.search(r'enqueued deposit for "scl_S[0-9]{4}-[0-9Xx-]{17}"', line)
        if match:
            queued_pids.append(match.group())

In [19]:
queued_pids

['enqueued deposit for "scl_S0001-37652018000301175"',
 'enqueued deposit for "scl_S0034-76122018000200321"',
 'enqueued deposit for "scl_S1516-44462018000200192"',
 'enqueued deposit for "scl_S0001-37652018000301035"',
 'enqueued deposit for "scl_S0034-76122018000200221"',
 'enqueued deposit for "scl_S1516-44462018000200227"',
 'enqueued deposit for "scl_S0001-37652018000300993"',
 'enqueued deposit for "scl_S0034-76122018000200264"',
 'enqueued deposit for "scl_S1516-44462018000200138"',
 'enqueued deposit for "scl_S0001-37652018000301215"',
 'enqueued deposit for "scl_S0102-33062018005006102"',
 'enqueued deposit for "scl_S1516-44462018000200220"',
 'enqueued deposit for "scl_S0001-37652018000301043"',
 'enqueued deposit for "scl_S0034-76122018000200244"',
 'enqueued deposit for "scl_S1516-44462018000200229"',
 'enqueued deposit for "scl_S0001-37652018000301073"',
 'enqueued deposit for "scl_S0102-33062018005006101"',
 'enqueued deposit for "scl_S1516-44462018000200212"',
 'enqueued

In [20]:
print('Total de PIDs enfileirados para depósito em 2018-07-27:', len(queued_pids))

Total de PIDs enfileirados para depósito em 2018-07-27: 5319


*Nota:* Durante a execução onde foram coletadas as mensagens de log sob análise, eu acompanhei o número de itens nas filas do *rabbitmq* com o comando *rabbitmqctl list_queues* e a fila de nome *celery* possuia milhares de itens (essa fila contém as tarefas à serem executadas pela task ``registry_dispatcher_document``, que é responsável por inserir o Depósito no pipeline), e esse número foi reduzindo rapidamente segundo à segundo.

In [21]:
with open('20180727-celeryworker.log') as celeryworker:
    registry_dispatcher_document_started_pids = []
    registry_dispatcher_document_finished_pids = []
    for line in celeryworker:
        match_start = re.search(r'''started the `registry_dispatcher_document` stage for "\('S[0-9]{4}-[0-9Xx-]{17}', 'scl'\)"''', line)
        if match_start:
            registry_dispatcher_document_started_pids.append(match_start.group())
        match_finish = re.search(r'''finished the `registry_dispatcher_document` stage for "\('S[0-9]{4}-[0-9Xx-]{17}', 'scl'\)"''', line)
        if match_finish:
            registry_dispatcher_document_finished_pids.append(match_finish.group())

In [22]:
print('Total de PIDs que passaram pelo estágio "registry_dispatcher_document":', len(registry_dispatcher_document_started_pids))
assert len(registry_dispatcher_document_started_pids) == len(registry_dispatcher_document_finished_pids)

Total de PIDs que passaram pelo estágio "registry_dispatcher_document": 122


Mensagens agrupadas por PID:

In [23]:
with open('20180727-celeryworker.log') as celeryworker:
    messages_by_pid = {}
    for line in celeryworker:
        match = re.search(r'S[0-9]{4}-[0-9Xx-]{17}', line)
        if match:
            _pid = messages_by_pid.setdefault(match.group(), [])
            _pid.append(line)

In [24]:
len(messages_by_pid)

122

O total de 122 grupos de mensagens por PID coincide com o total de PIDs que passaram pelo estágio "registry_dispatcher_document", o que faz sentido uma vez que trata-se do primeiro estágio.

Vamos agora listar o total de mensagens por grupo:

In [25]:
_freq = {}
for _pid, _msgs in messages_by_pid.items():
    print(_pid, len(_msgs))
    _occ = _freq.setdefault(len(_msgs), [])
    _occ.append(1)
    
_freq = {count: len(freq) for count, freq in _freq.items()}

S0001-37652018000301175 6
S1516-44462018000200192 6
S0034-76122018000200321 6
S0001-37652018000301035 6
S0034-76122018000200221 6
S0001-37652018000300993 6
S0034-76122018000200264 6
S1516-44462018000200227 6
S1516-44462018000200138 6
S0001-37652018000301215 6
S1516-44462018000200220 6
S0102-33062018005006102 6
S0001-37652018000301043 6
S0034-76122018000200244 6
S1516-44462018000200229 6
S0001-37652018000301073 6
S0102-33062018005006101 6
S1516-44462018000200212 6
S0001-37652018000300991 6
S0102-33062018005006103 6
S1516-44462018000200115 6
S0001-37652018000301187 6
S0034-76122018000200285 6
S1516-44462018000200226 6
S0001-37652018000301131 6
S0101-28002018005013104 6
S1516-44462018000200154 6
S0001-37652018000301251 6
S0102-33062018005006104 6
S1516-44462018000200145 6
S0001-37652018000301101 6
S0034-76122018000200199 6
S1516-44462018000200174 6
S0001-37652018000301233 6
S0034-76122018000200303 6
S1516-44462018000200181 6
S0001-37652018000301059 6
S1516-44462018000200163 6
S0001-376520

In [26]:
for _msg_no, _occ in _freq.items():
    print('ocorrências com', _msg_no, 'mensagens:\t', _occ)

ocorrências com 6 mensagens:	 118
ocorrências com 5 mensagens:	 4


Quais os PIDs que não foram processados?

In [27]:
non_processed_pids = set(pids) - set(messages_by_pid.keys())

In [28]:
non_processed_pids

{'S0102-09352018000200486',
 'S0103-636X2018000200749',
 'S2359-56472018000400323',
 'S1807-25772018000300175',
 'S1516-89132017000100450',
 'S0103-05822018000200238',
 'S0100-54052018000200127',
 'S0004-282X2018000700444',
 'S1413-99362018000100120',
 'S0102-86502018000500408',
 'S1519-69842018005016101',
 'S0104-12902018000200531',
 'S0103-05822018000200221',
 'S0037-86822018000300376',
 'S0103-636X2018000200459',
 'S0103-65132018000100211',
 'S1516-89132017000100459',
 'S1678-77572018000100488',
 'S0104-40362018005003004',
 'S0103-73312017000401125',
 'S1519-69842018000400796',
 'S1981-77462018000200643',
 'S1517-70762018000200550',
 'S1679-45082018000200100',
 'S0100-29452018000200904',
 'S0001-37652018000401459',
 'S1415-98482018000100204',
 'S2595-31922018000100009',
 'S0034-71672018000800905',
 'S1679-39512018000200218',
 'S1806-37132018005002101',
 'S2179-10742018000200217',
 'S1413-81232018000602051',
 'S1414-81452018000300705',
 'S0365-05962018000300426',
 'S1517-707620180002

In [29]:
len(non_processed_pids)

5197

In [30]:
with open('20180727-celeryworker.log') as exportdoi:
    received_tasks = []
    for line in exportdoi:
        match = re.search(r'Received task:.*$', line)
        if match:
            received_tasks.append(match.group())

In [31]:
received_tasks

['Received task: tasks.celery.registry_dispatcher_document[6cc167d4-b2d9-4027-9121-29dccf1ebfd1]  ',
 'Received task: tasks.celery.registry_dispatcher_document[093c19da-9af2-4f74-a1f6-7f2420f0bc44]  ',
 'Received task: tasks.celery.registry_dispatcher_document[bb72ec96-21f6-4a3a-b3e6-8efa17e3ed01]  ',
 'Received task: tasks.celery.registry_dispatcher_document[bc0bddc1-ca98-435e-843f-f86aa94d1cf4]  ',
 'Received task: tasks.celery.registry_dispatcher_document[30e6176c-50ba-4039-bc70-fbe718f7c0fc]  ',
 'Received task: tasks.celery.registry_dispatcher_document[27cc5d5f-cb83-47aa-860b-76c81265465a]  ',
 'Received task: tasks.celery.registry_dispatcher_document[fe44f48c-5087-456c-84b3-f55baedb7f99]  ',
 'Received task: tasks.celery.registry_dispatcher_document[0fa08ed5-0c01-4cf3-9da8-87582fd326d8]  ',
 'Received task: tasks.celery.registry_dispatcher_document[ed11b2c6-ac6a-4242-ba65-6222ffe01f28]  ',
 'Received task: tasks.celery.registry_dispatcher_document[6b286f72-71c3-4264-8c9d-f7d6a0fb

In [32]:
len(received_tasks)

122

## Conclusões

Nas duas análises, menos de 9% de todas as mensagens enfileiradas foram processadas pela task *registry_dispatcher_document*, que é responsável por colocar artigo no pipeline de depósito. As demais mensagens foram perdidas e suprimidas pela aplicação. É curioso como tanto o *Celery* quando o *RabbitMQ* não emitiram nenhuma mensagem informativa dando pistas do paradeiro de tantas mensagens. 

A solução se deu após deixarmos de utilizar o *RabbitMQ* em favor do *Redis*. Trata-se de uma ação ~~preguiçosa~~ pragmática - i.e., partirmos para outra solução ao invés de configurarmos corretamente o RabbitMQ -, mas que vai ao encontro dos objetivos de reduzirmos a variedade de softwares que utilizamos para resolver os mesmos problemas e simplificar o dia a dia da operação.

O processo de análise e debug resultou em uma série de melhorias no código, entre elas:
* https://github.com/scieloorg/doi_request/commit/ddf4839
* https://github.com/scieloorg/doi_request/commit/17663c5
* https://github.com/scieloorg/doi_request/commit/0d45930
* https://github.com/scieloorg/doi_request/commit/86cad20
* https://github.com/scieloorg/doi_request/commit/95d296a
* https://github.com/scieloorg/doi_request/commit/b6d3310
* https://github.com/scieloorg/doi_request/commit/9cea20f
* https://github.com/scieloorg/doi_request/commit/7dc28bd
* https://github.com/scieloorg/doi_request/commit/62d7fef
* https://github.com/scieloorg/doi_request/commit/f693659
* https://github.com/scieloorg/doi_request/commit/e8a1d36
* https://github.com/scieloorg/doi_request/commit/e579ba3
* https://github.com/scieloorg/doi_request/commit/889694e
* https://github.com/scieloorg/doi_request/commit/c6e4337
* https://github.com/scieloorg/doi_request/commit/13dc606
* https://github.com/scieloorg/doi_request/commit/6a97362
* https://github.com/scieloorg/doi_request/commit/c1b039e
* https://github.com/scieloorg/doi_request/commit/6df96f2
* https://github.com/scieloorg/doi_request/commit/8907374
* https://github.com/scieloorg/doi_request/commit/ad8b761
* https://github.com/scieloorg/doi_request/commit/03a3e2d
* https://github.com/scieloorg/doi_request/commit/6e8a954
* https://github.com/scieloorg/doi_request/commit/2b03843
* https://github.com/scieloorg/doi_request/commit/adf530b